# Descripción del proyecto

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Se tiene acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos. (los datos ya están procesado)


# Tabla de contenidos

* [1- Importación de librerias y carga de datos](#chapter1)
    * [Se cargan las librerías necesarias](#section_1_1)
    * [Se cargan los datos proporcionados](#section_1_2)
    
* [2 - Seleccion del modelo](#chapter2)

    * [Se segmentan los datos en tres conjuntos de entrenamiento, validación y prueba](#section_2_1)
    * [Se evalúa la calidad de los diferentes modelos de clasificación: se entrenarán los modelos con el conjunto de datos de entrenamiento y se calculará su nivel de exactitud (accuracy) con el conjunto de validación](#section_2_2)
       -  Modelo árbol de decisión: utilizando un ciclo FOR, se buscará el valor de profundidad del arbol que permita obtener el mayor valor accuracy
       - Modelo bosque aleatorio: utilizando dos ciclos FOR, se buscará el valor de profundidad del árbol y la cantidad de estimadores que permitan obtener el mayor valor accuracy
       - Regresion logística: se calculará el valor del score para el modelo  
    * [Se selecciona el modelo con mayor exactitud y se evalúa su calidad utilizando el conjunto de prueba](#section_2_3)
    
    
* [3- Prueba de cordura](#chapter3)
    
     
* [4- Conclusiones](#chapter4)

# 1- Importación de librerias y carga de datos <a class="anchor" id="chapter1"></a>


 1_1 Importación de las librerias <a class="anchor" id="section_1_1"></a>
 

In [1]:
#Se importan las librerias

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


 1_2 Carga y analisis prelimiar de los datos <a class="anchor" id="section_1_2"></a>

In [2]:
#Se cargan los datos

df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Los datos proporcionados cuentan con 5 variables segun se describe a continuación:

- сalls: número de llamadas,
- minutes: duración total de la llamada en minutos,
- messages: número de mensajes de texto,
- mb_used: Tráfico de Internet utilizado en MB,
- is_ultra: plan para el mes actual (Ultra - 1, Smart - 0).


# 2 - Selección del modelo <a class="anchor" id="chapter2"></a>

 2_1 Segmentacion de datos <a class="anchor" id="section_2_1"></a>

In [5]:
# Se segmentan los datos en tres conjuntos (entrenamiento (60%), validacion (20%)y prueba (20%)). El tipo de plan es la variable objetivo

features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size = 0.4, random_state= 12345)
features_test, features_valid, target_test, target_valid  =train_test_split(features_valid, target_valid, test_size = 0.5, random_state= 12345)


2_2 Análisis y selección del modelo <a class="anchor" id="section_2_2"></a>

* MODELO ARBOL DE DECISION. 

Se busca el valor del hiperparametro "max_depth" que permita obtener el mayor valor de exactitud

In [6]:
# Evaluacion Modelo Arbol de decision. 

best_model_decision_tree = None
best_result_decision_tree = 0
best_depth_decision_tree  = 0
for depth in range(1, 12):
    model = DecisionTreeClassifier(random_state=12345, max_depth= depth) #se crea el modelo con la profundidad depth
    model.fit(features_train, target_train) # se entrena el modelo 
    predictions = model.predict(features_valid) # se obtienen las predicciones del modelo
    result = accuracy_score(target_valid, predictions) # se calcula la exactitud
    if result > best_result_decision_tree:
        best_model_decision_tree = model # guarda el modelo que corresponde a la mejor puntuacion de accuracy
        best_result_decision_tree = result # guarda la mejor puntuacion de accuracy
        best_depth_decision_tree = depth # guarda la profundidad que corresponde a la mejor puntuacion de accuracy

print("Mejor modelo 'Arbol de decision':")
print("Exactitud:", best_result_decision_tree, "", "Profundidad:",  best_depth_decision_tree )


Mejor modelo 'Arbol de decision':
Exactitud: 0.7993779160186625  Profundidad: 7


* MODELO BOSQUE ALEATORIO

Se busca el valor de los hiperparametros "max_depth" y "n_estimators" que permitan obtener el mayor valor de exactitud

In [7]:
#Evaluacion Modelo de bosque aleatorio

best_score_random_forest = 0
best_est_random_forest = 0
best_depth_random_forest = 0
for est in range(1, 14): # selecciona el rango del hiperparámetro
    for depth in range(1,12):
        model = RandomForestClassifier(random_state=54321, max_depth= depth, n_estimators=est) # configura el número de árboles
        model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
        score = model.score(features_valid, target_valid) # calcula la puntuación de accuracy en el conjunto de validación
        if score > best_score_random_forest:
            best_score_random_forest = score# guarda la mejor puntuación de accuracy en el conjunto de validación
            best_est_random_forest = est# guarda el número de estimadores que corresponden a la mejor puntuación de accuracy
            best_depth_random_forest = depth # guarda la la profundidad que corresponde a la mejor puntacion den accuracy

print("Mejor modelo 'Bosque Aleatorio':")
print("Exactitud:", best_score_random_forest, "", "Profundidad:",  best_depth_random_forest, "n_estimators:", best_est_random_forest)




Mejor modelo 'Bosque Aleatorio':
Exactitud: 0.8133748055987559  Profundidad: 9 n_estimators: 6


* MODELO REGRESION LOGISTICA

    - Se calcula el valor de accuracy para el modelo 

In [8]:
#Evaluacion Regresion logistica


model = LogisticRegression(random_state=54321, solver='liblinear')  # se inicializa el constructor de regresión logística con los parámetros random_state=54321 y solver='liblinear'
model.fit(features_train, target_train) # se entrena el modelo en el conjunto de entrenamiento
score_valid = model.score(features_valid, target_valid) # se calcula la puntuación de accuracy en el conjunto de validación

print("Exactitud del modelo de regresión logística en el conjunto de validación:", score_valid)



Exactitud del modelo de regresión logística en el conjunto de validación: 0.7402799377916018


- El modelo de arbol de decision y el de random forest arrojan un valor de exactitud muy similar de 0,8. Dado que el modelo de arbol de decision es mas sencillo, optamos por este ultimo para predecir los valores deseados

2_3 Comprobación de la calidad del modelo <a class="anchor" id="section_2_3"></a>

    - Se comprueba la calidad del modelo seleccionado utilizando el conjunto de prueba

In [9]:
# Se comprueba la calidad del modelo seleccionado utilizando el conjunto de prueba

model = DecisionTreeClassifier(random_state=12345, max_depth= best_depth_decision_tree) # se crea un modelo con la profundidad proporcionada
model.fit(features_train, target_train) # se entrena el modelo 
predictions = model.predict(features_test) # se obtienen las predicciones del modelo para el conjunto de prueba
result = accuracy_score(target_test, predictions) #  se calcula la exactitud
print(result)

0.7822706065318819


- La exactitud del modelo seleccionado para el conjunto de prieba es de 0,78. Dicho valor se encuentra por encima del umbral de 0,75 solicitado.




# 3 - Prueba de cordura <a class="anchor" id="chapter3"></a>

In [10]:
#Prueba de cordura

predictions = pd.Series(np.random.choice([0, 1], size=len(target_test)), index=target_test.index)

score = model.score(features_test, predictions)
print("Exactitud de prueba de cordura:", score)


Exactitud de prueba de cordura: 0.4665629860031104


- La prueba de cordura arroja un valor de exactitud inferior al obtenido con el modelo de árbol de decisión seleccionado, por lo tanto se concluye que el modelo cuenta con una calidad adecuada para estimar los parametros ya que es superior a la de asignar valores 1 y 0 al azar




# 4 - Conclusiones <a class="anchor" id="chapter4"></a>

- Al evaluar la exactitud de los modelos analizados, el árbol de decisión y el bosque aleatorio arrojaron un valor muy similar en torno a 0,8 y superior al de la regresión logística. 

- Dado que el modelo de árbol de decisión es mas sencillo que el de bosque aleatorio, optamos por este último para predecir los valores deseados 

- La profundidad del arbol de decisión para el modelo con mayor exactitud resulto en un valor de 7

- Al comprobar la calidad del modelo en el conjunto de prueba se obtuvo un valore de exactitud del modelo seleccionado de 0,78. Dicho valor se encuentra por encima del umbral de 0,75 solicitado.

- Prueba de cordura del modelo seleccionado. Utilizando como valores de predicción unos y ceros aleatoriamente, se obtuvo un valor de exactirud de 0,51. Por lo tanto el modelo seleccionado paso la prueba ya que su valor de exactitud es superior al de la prueba de cordura.


    